In [1]:
# Dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [2]:
# Ininital test using AMPY
# Load data
data = pd.read_csv('Data/AMPY_cleaned.csv').set_index('Date')

data.head()

,Open,High,Low,Close,Volume,Day Result
Date,,,,,,
2017-01-03,20.420000,21.24,20.100000,21.000000,7800,1
2017-01-04,20.850000,21.00,20.700001,20.990000,5500,1
2017-01-05,21.000000,21.25,20.760000,21.250000,5800,1
2017-01-06,21.430000,21.43,20.750000,21.190001,14100,0
2017-01-09,21.129999,21.85,20.500000,21.139999,25000,1


In [6]:
# Use the open price from 2017-01-01 to 2022-04-30 to train the model
training_set = data.loc['2017-01-01':'2022-04-30'].reset_index()
training_set = training_set.iloc[:, 1:2].values


print(training_set)
print(training_set.shape)
print(type(training_set))

[[20.42000008]
 [20.85000038]
 [21.        ]
 ...
 [ 6.5       ]
 [ 6.76999998]
 [ 6.92000008]]
(1341, 1)
<class 'numpy.ndarray'>


In [7]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

training_set_scaled

array([[0.95117279],
       [0.97175681],
       [0.97893727],
       ...,
       [0.28482527],
       [0.29775011],
       [0.30493058]])

In [8]:
# Create the data structure
X_train = []
y_train = []
for i in range(60, 1341):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
# Reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_train.shape

(1281, 60, 1)

In [10]:
# Build the model
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [11]:
# Fit the model
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
41/41 [==============================] - 6s 57ms/step - loss: 0.0272
Epoch 2/100
41/41 [==============================] - 2s 56ms/step - loss: 0.0056
Epoch 3/100
41/41 [==============================] - 2s 53ms/step - loss: 0.0047
Epoch 4/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0042
Epoch 5/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0042
Epoch 6/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0039
Epoch 7/100
41/41 [==============================] - 2s 53ms/step - loss: 0.0036
Epoch 8/100
41/41 [==============================] - 2s 53ms/step - loss: 0.0036
Epoch 9/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0031
Epoch 10/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0037
Epoch 11/100
41/41 [==============================] - 2s 52ms/step - loss: 0.0033
Epoch 12/100
41/41 [==============================] - 2s 53ms/step - loss: 0.0036
Epoch 13/100
41/41 [=====

In [12]:
# Extract the actual stock price for May 2022
actual_stock_price = data.loc['2022-05-01':'2022-05-31'].reset_index()
actual_stock_price = actual_stock_price.iloc[:, 1:2].values

print(actual_stock_price)
print(actual_stock_price.shape)
print(type(actual_stock_price))


[[6.53000021]
 [6.69000006]
 [7.26999998]
 [7.53000021]
 [7.5       ]
 [7.30999994]
 [6.65999985]
 [6.51000023]
 [6.19999981]
 [6.78000021]
 [6.98000002]
 [7.48999977]]
(12, 1)
<class 'numpy.ndarray'>


In [20]:
# Prepare input for model
dataset_total = data['Open']
inputs = dataset_total[len(dataset_total) - len(actual_stock_price) - 60:].values

inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)

print(X_test.shape)
print(X_test.shape[0])
print(X_test.shape[1])

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

(20,)
20


C:\Users\Maks\AppData\Local\Temp\ipykernel_34804\2518568727.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)


IndexError: tuple index out of range